# Flight Delay Insurance Model - Experiment Log

## Overview
This notebook tracks all experiments for optimizing the flight delay prediction model.

**Goal**: Minimize Custom Payout MSE (Currently ~3684, Baseline ~4677)

**Evaluation Metric**: Mean Squared Error between actual payout and expected payout

---


## Setup: Load Data and Define Evaluation Framework


In [1]:
import kagglehub
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, log_loss, classification_report

print("Libraries imported successfully")


Libraries imported successfully


In [2]:
# Load dataset
path_dir = kagglehub.dataset_download("shubhamsingh42/flight-delay-dataset-2018-2024")
file_name = "flight_data_2018_2024.csv"
path_to_file = os.path.join(path_dir, file_name)
df = pd.read_csv(path_to_file, low_memory=False)

print(f"Dataset loaded: {df.shape}")


Dataset loaded: (582425, 120)


In [3]:
# Data cleaning (from main notebook)
input_features = [
    'Marketing_Airline_Network', 'Quarter', 'Month', 'DayofMonth',
    'DayOfWeek', 'CRSDepTime', 'OriginAirportID', 'DestAirportID',
    'OriginCityMarketID', 'DestCityMarketID', 'Distance'
]

target_features = [
    'ArrDelayMinutes', 'Cancelled', 'Diverted'
]
df_clean = df[input_features + target_features].copy()

bin1 = 60
bin2 = 120

# Create delay categories
df_clean['Delay_Category'] = -1
df_clean.loc[(df_clean['Cancelled'] == 1) | (df_clean['Diverted'] == 1), 'Delay_Category'] = 4
df_clean.loc[(df_clean['Delay_Category'] != 4) & (df_clean['ArrDelayMinutes'] >= bin2), 'Delay_Category'] = 3
df_clean.loc[(df_clean['Delay_Category'] != 4) & (df_clean['ArrDelayMinutes'] >= bin1) & (df_clean['ArrDelayMinutes'] < bin2), 'Delay_Category'] = 2
df_clean.loc[(df_clean['Delay_Category'] != 4) & (df_clean['ArrDelayMinutes'] > 0) & (df_clean['ArrDelayMinutes'] < bin1), 'Delay_Category'] = 1
df_clean.loc[(df_clean['Delay_Category'] != 4) & (df_clean['ArrDelayMinutes'] == 0), 'Delay_Category'] = 0
df_clean = df_clean[df_clean['Delay_Category'] != -1]

df_clean = df_clean.drop(columns=target_features)

CATEGORICAL_FEATURES = [
    'Marketing_Airline_Network', 'Quarter', 'Month', 'DayofMonth',
    'DayOfWeek', 'OriginAirportID', 'DestAirportID', 'OriginCityMarketID',
    'DestCityMarketID'
]

for col in CATEGORICAL_FEATURES:
    df_clean[col] = df_clean[col].astype('category')

def time_to_block(time_hhmm):
    hour = time_hhmm // 100
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

df_clean['CRSDepTime_Block'] = df_clean['CRSDepTime'].apply(time_to_block).astype('category')
df_clean['CRSDepTime'] = df_clean['CRSDepTime'] // 100 * 60 + df_clean['CRSDepTime'] % 100

print("Data cleaning completed")
print(f"Final shape: {df_clean.shape}")
print(f"\nDelay category distribution:\n{df_clean['Delay_Category'].value_counts().sort_index()}")


Data cleaning completed
Final shape: (582425, 13)

Delay category distribution:
Delay_Category
0    336861
1    172087
2     28170
3     21597
4     23710
Name: count, dtype: int64


## Evaluation Framework


In [4]:
# Constants
NUM_CLASSES = 5
PAYOUT_MAP = {
    0: 0,
    1: 50,
    2: 100,
    3: 300,
    4: 200
}
N_SPLITS = 3
RANDOM_STATE = 42

def calculate_expected_payout_mse(y_true_categories, y_proba):
    """
    Calculates the Mean Squared Error between the Actual Payout and the Expected Payout.
    """
    y_actual_payout = y_true_categories.map(PAYOUT_MAP).values
    payout_vector = np.array([PAYOUT_MAP[i] for i in range(NUM_CLASSES)])
    y_expected_payout = np.dot(y_proba, payout_vector)
    custom_mse = mean_squared_error(y_actual_payout, y_expected_payout)
    return custom_mse, y_actual_payout, y_expected_payout

# Cross-validation strategy
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

print("Evaluation framework defined")


Evaluation framework defined


In [5]:
# Experiment tracking
experiment_results = []

def log_experiment(experiment_id, model_name, rationale, parameters, cv_scores, mean_score, std_score, additional_notes=""):
    """
    Log experiment results to tracking list
    """
    result = {
        'Experiment_ID': experiment_id,
        'Model': model_name,
        'Rationale': rationale,
        'Parameters': str(parameters),
        'CV_Scores': cv_scores,
        'Mean_MSE': mean_score,
        'Std_MSE': std_score,
        'Notes': additional_notes,
        'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    experiment_results.append(result)
    return result

def show_experiment_summary():
    """
    Display summary of all experiments
    """
    df_results = pd.DataFrame(experiment_results)
    df_summary = df_results[['Experiment_ID', 'Model', 'Mean_MSE', 'Std_MSE', 'Rationale']].copy()
    df_summary = df_summary.sort_values('Mean_MSE')
    return df_summary

print("Experiment logging functions defined")


Experiment logging functions defined


## Baseline: Current Best Model (XGBoost from main notebook)


In [ ]:
from xgboost import XGBClassifier

# Prepare baseline data
X_baseline = df_clean.drop(columns=['Delay_Category'])
y_baseline = df_clean['Delay_Category']

NUMERICAL_FEATURES = ['Distance', 'CRSDepTime']
CATEGORICAL_FEATURES_BASE = X_baseline.select_dtypes(include='category').columns.tolist()

preprocessor_baseline = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
             ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
             ('scaler', StandardScaler())]), NUMERICAL_FEATURES),
        ('cat', OneHotEncoder(handle_unknown='ignore'), CATEGORICAL_FEATURES_BASE)
    ], remainder='drop'
)

xgb_baseline = XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    n_estimators=300,
    max_depth=7,
    learning_rate=0.05,
    num_class=NUM_CLASSES,
    reg_lambda=1,
    use_label_encoder=False,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    tree_method='hist'
)

# Run baseline cross-validation
print("Running Baseline XGBoost...")
cv_scores_baseline = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_baseline, y_baseline), 1):
    X_train, X_test = X_baseline.iloc[train_index], X_baseline.iloc[test_index]
    y_train, y_test = y_baseline.iloc[train_index], y_baseline.iloc[test_index]
    
    X_train_processed = preprocessor_baseline.fit_transform(X_train)
    X_test_processed = preprocessor_baseline.transform(X_test)
    
    xgb_baseline.fit(X_train_processed, y_train, verbose=False)
    y_pred_proba = xgb_baseline.predict_proba(X_test_processed)
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
    cv_scores_baseline.append(custom_mse)
    print(f"  Fold {fold_idx}: MSE = {custom_mse:.2f}")

mean_mse_baseline = np.mean(cv_scores_baseline)
std_mse_baseline = np.std(cv_scores_baseline)

print(f"\nBaseline Mean MSE: {mean_mse_baseline:.2f} ± {std_mse_baseline:.2f}")

# Log baseline
log_experiment(
    experiment_id="BASELINE",
    model_name="XGBoost (Original)",
    rationale="Current best model from main notebook - establishes benchmark",
    parameters=xgb_baseline.get_params(),
    cv_scores=cv_scores_baseline,
    mean_score=mean_mse_baseline,
    std_score=std_mse_baseline,
    additional_notes="Uses basic features with log1p transform on numerical features"
)


Running Baseline XGBoost...


In [ ]:
# Create a copy for feature engineering
df_features = df_clean.copy()
print(f"Starting feature engineering with base shape: {df_features.shape}")


## Experiment A1: Temporal Features - Weekend Indicator

**Rationale**: Weekends may have different delay patterns (less business travel, different staffing)


In [ ]:
# Add weekend indicator (DayOfWeek: 1=Monday, ..., 7=Sunday)
df_features['IsWeekend'] = (df_features['DayOfWeek'].isin([6, 7])).astype('category')

# Test with baseline XGBoost
X_a1 = df_features.drop(columns=['Delay_Category'])
y_a1 = df_features['Delay_Category']

CATEGORICAL_FEATURES_A1 = X_a1.select_dtypes(include='category').columns.tolist()

preprocessor_a1 = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
             ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
             ('scaler', StandardScaler())]), NUMERICAL_FEATURES),
        ('cat', OneHotEncoder(handle_unknown='ignore'), CATEGORICAL_FEATURES_A1)
    ], remainder='drop'
)

print("Experiment A1: Weekend Indicator")
cv_scores_a1 = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_a1, y_a1), 1):
    X_train, X_test = X_a1.iloc[train_index], X_a1.iloc[test_index]
    y_train, y_test = y_a1.iloc[train_index], y_a1.iloc[test_index]
    
    X_train_processed = preprocessor_a1.fit_transform(X_train)
    X_test_processed = preprocessor_a1.transform(X_test)
    
    model = XGBClassifier(**xgb_baseline.get_params())
    model.fit(X_train_processed, y_train, verbose=False)
    y_pred_proba = model.predict_proba(X_test_processed)
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
    cv_scores_a1.append(custom_mse)

mean_mse_a1 = np.mean(cv_scores_a1)
std_mse_a1 = np.std(cv_scores_a1)

print(f"Mean MSE: {mean_mse_a1:.2f} ± {std_mse_a1:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - mean_mse_a1:.2f}")

log_experiment(
    experiment_id="A1",
    model_name="XGBoost + Weekend",
    rationale="Weekend flights may have different delay patterns",
    parameters={"added_features": "IsWeekend"},
    cv_scores=cv_scores_a1,
    mean_score=mean_mse_a1,
    std_score=std_mse_a1,
    additional_notes=f"Improvement: {mean_mse_baseline - mean_mse_a1:.2f}"
)


## Experiment A2: Temporal Features - Rush Hour Indicator

**Rationale**: Morning (6-9am) and evening (4-7pm) rush hours may have more congestion and delays


In [ ]:
# Add rush hour indicator
def is_rush_hour(time_minutes):
    hour = time_minutes // 60
    # Morning rush: 6-9am, Evening rush: 4-7pm (16-19)
    return ((6 <= hour < 9) | (16 <= hour < 19))

df_features['IsRushHour'] = df_features['CRSDepTime'].apply(is_rush_hour).astype('category')

X_a2 = df_features.drop(columns=['Delay_Category'])
y_a2 = df_features['Delay_Category']

CATEGORICAL_FEATURES_A2 = X_a2.select_dtypes(include='category').columns.tolist()

preprocessor_a2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
             ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
             ('scaler', StandardScaler())]), NUMERICAL_FEATURES),
        ('cat', OneHotEncoder(handle_unknown='ignore'), CATEGORICAL_FEATURES_A2)
    ], remainder='drop'
)

print("Experiment A2: Rush Hour Indicator")
cv_scores_a2 = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_a2, y_a2), 1):
    X_train, X_test = X_a2.iloc[train_index], X_a2.iloc[test_index]
    y_train, y_test = y_a2.iloc[train_index], y_a2.iloc[test_index]
    
    X_train_processed = preprocessor_a2.fit_transform(X_train)
    X_test_processed = preprocessor_a2.transform(X_test)
    
    model = XGBClassifier(**xgb_baseline.get_params())
    model.fit(X_train_processed, y_train, verbose=False)
    y_pred_proba = model.predict_proba(X_test_processed)
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
    cv_scores_a2.append(custom_mse)

mean_mse_a2 = np.mean(cv_scores_a2)
std_mse_a2 = np.std(cv_scores_a2)

print(f"Mean MSE: {mean_mse_a2:.2f} ± {std_mse_a2:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - mean_mse_a2:.2f}")

log_experiment(
    experiment_id="A2",
    model_name="XGBoost + Rush Hour",
    rationale="Rush hour flights face more airport congestion",
    parameters={"added_features": "IsWeekend, IsRushHour"},
    cv_scores=cv_scores_a2,
    mean_score=mean_mse_a2,
    std_score=std_mse_a2,
    additional_notes=f"Improvement: {mean_mse_baseline - mean_mse_a2:.2f}"
)


## Experiment B1: Route Features - Distance Bins

**Rationale**: Short, medium, and long-haul flights have different operational characteristics and delay patterns


In [ ]:
# Add distance bins
def categorize_distance(distance):
    if distance < 500:
        return 'Short'  # <500 miles
    elif distance < 1500:
        return 'Medium'  # 500-1500 miles
    else:
        return 'Long'  # >1500 miles

df_features['Distance_Bin'] = df_features['Distance'].apply(categorize_distance).astype('category')

X_b1 = df_features.drop(columns=['Delay_Category'])
y_b1 = df_features['Delay_Category']

CATEGORICAL_FEATURES_B1 = X_b1.select_dtypes(include='category').columns.tolist()

preprocessor_b1 = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
             ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
             ('scaler', StandardScaler())]), NUMERICAL_FEATURES),
        ('cat', OneHotEncoder(handle_unknown='ignore'), CATEGORICAL_FEATURES_B1)
    ], remainder='drop'
)

print("Experiment B1: Distance Bins")
cv_scores_b1 = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_b1, y_b1), 1):
    X_train, X_test = X_b1.iloc[train_index], X_b1.iloc[test_index]
    y_train, y_test = y_b1.iloc[train_index], y_b1.iloc[test_index]
    
    X_train_processed = preprocessor_b1.fit_transform(X_train)
    X_test_processed = preprocessor_b1.transform(X_test)
    
    model = XGBClassifier(**xgb_baseline.get_params())
    model.fit(X_train_processed, y_train, verbose=False)
    y_pred_proba = model.predict_proba(X_test_processed)
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
    cv_scores_b1.append(custom_mse)

mean_mse_b1 = np.mean(cv_scores_b1)
std_mse_b1 = np.std(cv_scores_b1)

print(f"Mean MSE: {mean_mse_b1:.2f} ± {std_mse_b1:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - mean_mse_b1:.2f}")

log_experiment(
    experiment_id="B1",
    model_name="XGBoost + Distance Bins",
    rationale="Different flight distances have different delay characteristics",
    parameters={"added_features": "IsWeekend, IsRushHour, Distance_Bin"},
    cv_scores=cv_scores_b1,
    mean_score=mean_mse_b1,
    std_score=std_mse_b1,
    additional_notes=f"Improvement: {mean_mse_baseline - mean_mse_b1:.2f}"
)


## Experiment C1: Historical Features - Airport Delay Rates

**Rationale**: Some airports have consistently higher delay rates; calculate aggregated statistics


In [ ]:
# Calculate origin and destination airport delay rates
# Use the full dataset to avoid data leakage (we're using historical aggregate stats)
origin_delay_rate = df_features.groupby('OriginAirportID')['Delay_Category'].apply(lambda x: (x > 0).mean())
dest_delay_rate = df_features.groupby('DestAirportID')['Delay_Category'].apply(lambda x: (x > 0).mean())

df_features['Origin_Delay_Rate'] = df_features['OriginAirportID'].map(origin_delay_rate)
df_features['Dest_Delay_Rate'] = df_features['DestAirportID'].map(dest_delay_rate)

# Update numerical features
NUMERICAL_FEATURES_C1 = ['Distance', 'CRSDepTime', 'Origin_Delay_Rate', 'Dest_Delay_Rate']

X_c1 = df_features.drop(columns=['Delay_Category'])
y_c1 = df_features['Delay_Category']

CATEGORICAL_FEATURES_C1 = X_c1.select_dtypes(include='category').columns.tolist()

preprocessor_c1 = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
             ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
             ('scaler', StandardScaler())]), NUMERICAL_FEATURES_C1),
        ('cat', OneHotEncoder(handle_unknown='ignore'), CATEGORICAL_FEATURES_C1)
    ], remainder='drop'
)

print("Experiment C1: Airport Delay Rates")
cv_scores_c1 = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_c1, y_c1), 1):
    X_train, X_test = X_c1.iloc[train_index], X_c1.iloc[test_index]
    y_train, y_test = y_c1.iloc[train_index], y_c1.iloc[test_index]
    
    X_train_processed = preprocessor_c1.fit_transform(X_train)
    X_test_processed = preprocessor_c1.transform(X_test)
    
    model = XGBClassifier(**xgb_baseline.get_params())
    model.fit(X_train_processed, y_train, verbose=False)
    y_pred_proba = model.predict_proba(X_test_processed)
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
    cv_scores_c1.append(custom_mse)

mean_mse_c1 = np.mean(cv_scores_c1)
std_mse_c1 = np.std(cv_scores_c1)

print(f"Mean MSE: {mean_mse_c1:.2f} ± {std_mse_c1:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - mean_mse_c1:.2f}")

log_experiment(
    experiment_id="C1",
    model_name="XGBoost + Airport Delay Rates",
    rationale="Historical airport performance predicts future delays",
    parameters={"added_features": "All previous + Origin_Delay_Rate, Dest_Delay_Rate"},
    cv_scores=cv_scores_c1,
    mean_score=mean_mse_c1,
    std_score=std_mse_c1,
    additional_notes=f"Improvement: {mean_mse_baseline - mean_mse_c1:.2f}"
)


## Experiment C2: Historical Features - Airline Delay Rates

**Rationale**: Different airlines have different operational efficiency and delay rates


In [ ]:
# Calculate airline delay rates
airline_delay_rate = df_features.groupby('Marketing_Airline_Network')['Delay_Category'].apply(lambda x: (x > 0).mean())
df_features['Airline_Delay_Rate'] = df_features['Marketing_Airline_Network'].map(airline_delay_rate)

NUMERICAL_FEATURES_C2 = ['Distance', 'CRSDepTime', 'Origin_Delay_Rate', 'Dest_Delay_Rate', 'Airline_Delay_Rate']

X_c2 = df_features.drop(columns=['Delay_Category'])
y_c2 = df_features['Delay_Category']

CATEGORICAL_FEATURES_C2 = X_c2.select_dtypes(include='category').columns.tolist()

preprocessor_c2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
             ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
             ('scaler', StandardScaler())]), NUMERICAL_FEATURES_C2),
        ('cat', OneHotEncoder(handle_unknown='ignore'), CATEGORICAL_FEATURES_C2)
    ], remainder='drop'
)

print("Experiment C2: Airline Delay Rates")
cv_scores_c2 = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_c2, y_c2), 1):
    X_train, X_test = X_c2.iloc[train_index], X_c2.iloc[test_index]
    y_train, y_test = y_c2.iloc[train_index], y_c2.iloc[test_index]
    
    X_train_processed = preprocessor_c2.fit_transform(X_train)
    X_test_processed = preprocessor_c2.transform(X_test)
    
    model = XGBClassifier(**xgb_baseline.get_params())
    model.fit(X_train_processed, y_train, verbose=False)
    y_pred_proba = model.predict_proba(X_test_processed)
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
    cv_scores_c2.append(custom_mse)

mean_mse_c2 = np.mean(cv_scores_c2)
std_mse_c2 = np.std(cv_scores_c2)

print(f"Mean MSE: {mean_mse_c2:.2f} ± {std_mse_c2:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - mean_mse_c2:.2f}")

log_experiment(
    experiment_id="C2",
    model_name="XGBoost + Airline Delay Rates",
    rationale="Airline operational efficiency is a strong predictor",
    parameters={"added_features": "All previous + Airline_Delay_Rate"},
    cv_scores=cv_scores_c2,
    mean_score=mean_mse_c2,
    std_score=std_mse_c2,
    additional_notes=f"Improvement: {mean_mse_baseline - mean_mse_c2:.2f}"
)


## Feature Engineering Summary


In [ ]:
# Show feature engineering results
print("="*70)
print("FEATURE ENGINEERING SUMMARY")
print("="*70)
feature_summary = show_experiment_summary()
print(feature_summary.to_string(index=False))
print("="*70)

# Determine best feature set
best_features_exp = feature_summary.iloc[0]
print(f"\nBest Feature Set: {best_features_exp['Experiment_ID']} - {best_features_exp['Model']}")
print(f"Best MSE: {best_features_exp['Mean_MSE']:.2f}")
print(f"Improvement over baseline: {mean_mse_baseline - best_features_exp['Mean_MSE']:.2f}")


---
# Phase 3: Model Architecture Exploration

**Strategy**: Test different model families with the best feature set


In [ ]:
# Use the best feature set from previous phase
# For now, we'll use all engineered features (C2)
X_best = df_features.drop(columns=['Delay_Category'])
y_best = df_features['Delay_Category']

NUMERICAL_FEATURES_BEST = ['Distance', 'CRSDepTime', 'Origin_Delay_Rate', 'Dest_Delay_Rate', 'Airline_Delay_Rate']
CATEGORICAL_FEATURES_BEST = X_best.select_dtypes(include='category').columns.tolist()

print(f"Best feature set: {len(NUMERICAL_FEATURES_BEST)} numerical + {len(CATEGORICAL_FEATURES_BEST)} categorical")
print(f"Total samples: {len(X_best)}")


## Experiment D1: LightGBM Model

**Rationale**: LightGBM is faster than XGBoost and often performs better with categorical features


In [ ]:
from lightgbm import LGBMClassifier

preprocessor_best = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
             ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
             ('scaler', StandardScaler())]), NUMERICAL_FEATURES_BEST),
        ('cat', OneHotEncoder(handle_unknown='ignore'), CATEGORICAL_FEATURES_BEST)
    ], remainder='drop'
)

lgbm_model = LGBMClassifier(
    objective='multiclass',
    num_class=NUM_CLASSES,
    n_estimators=300,
    max_depth=7,
    learning_rate=0.05,
    reg_lambda=1,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=-1
)

print("Experiment D1: LightGBM")
cv_scores_d1 = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_best, y_best), 1):
    X_train, X_test = X_best.iloc[train_index], X_best.iloc[test_index]
    y_train, y_test = y_best.iloc[train_index], y_best.iloc[test_index]
    
    X_train_processed = preprocessor_best.fit_transform(X_train)
    X_test_processed = preprocessor_best.transform(X_test)
    
    lgbm_model.fit(X_train_processed, y_train)
    y_pred_proba = lgbm_model.predict_proba(X_test_processed)
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
    cv_scores_d1.append(custom_mse)
    print(f"  Fold {fold_idx}: MSE = {custom_mse:.2f}")

mean_mse_d1 = np.mean(cv_scores_d1)
std_mse_d1 = np.std(cv_scores_d1)

print(f"\nMean MSE: {mean_mse_d1:.2f} ± {std_mse_d1:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - mean_mse_d1:.2f}")

log_experiment(
    experiment_id="D1",
    model_name="LightGBM",
    rationale="LightGBM often outperforms XGBoost with better categorical handling",
    parameters=lgbm_model.get_params(),
    cv_scores=cv_scores_d1,
    mean_score=mean_mse_d1,
    std_score=std_mse_d1,
    additional_notes=f"Improvement: {mean_mse_baseline - mean_mse_d1:.2f}"
)


## Experiment D2: CatBoost Model

**Rationale**: CatBoost has native categorical feature support and strong baseline performance


In [ ]:
from catboost import CatBoostClassifier

# CatBoost can handle categorical features natively
catboost_model = CatBoostClassifier(
    iterations=300,
    depth=7,
    learning_rate=0.05,
    loss_function='MultiClass',
    random_state=RANDOM_STATE,
    verbose=False,
    thread_count=-1
)

print("Experiment D2: CatBoost")
cv_scores_d2 = []

# Get categorical feature indices for CatBoost
cat_feature_indices = [i for i, col in enumerate(X_best.columns) if col in CATEGORICAL_FEATURES_BEST]

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_best, y_best), 1):
    X_train, X_test = X_best.iloc[train_index], X_best.iloc[test_index]
    y_train, y_test = y_best.iloc[train_index], y_best.iloc[test_index]
    
    # CatBoost preprocessor (only numerical features need transformation)
    preprocessor_catboost = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                 ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
                 ('scaler', StandardScaler())]), NUMERICAL_FEATURES_BEST)
        ], remainder='passthrough'
    )
    
    X_train_processed = preprocessor_catboost.fit_transform(X_train)
    X_test_processed = preprocessor_catboost.transform(X_test)
    
    # Identify new categorical indices after preprocessing
    num_numerical = len(NUMERICAL_FEATURES_BEST)
    cat_indices_processed = list(range(num_numerical, X_train_processed.shape[1]))
    
    catboost_model.fit(X_train_processed, y_train, cat_features=cat_indices_processed)
    y_pred_proba = catboost_model.predict_proba(X_test_processed)
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
    cv_scores_d2.append(custom_mse)
    print(f"  Fold {fold_idx}: MSE = {custom_mse:.2f}")

mean_mse_d2 = np.mean(cv_scores_d2)
std_mse_d2 = np.std(cv_scores_d2)

print(f"\nMean MSE: {mean_mse_d2:.2f} ± {std_mse_d2:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - mean_mse_d2:.2f}")

log_experiment(
    experiment_id="D2",
    model_name="CatBoost",
    rationale="Native categorical handling may improve performance",
    parameters=catboost_model.get_params(),
    cv_scores=cv_scores_d2,
    mean_score=mean_mse_d2,
    std_score=std_mse_d2,
    additional_notes=f"Improvement: {mean_mse_baseline - mean_mse_d2:.2f}"
)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=0
)

print("Experiment D3: Random Forest")
cv_scores_d3 = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_best, y_best), 1):
    X_train, X_test = X_best.iloc[train_index], X_best.iloc[test_index]
    y_train, y_test = y_best.iloc[train_index], y_best.iloc[test_index]
    
    X_train_processed = preprocessor_best.fit_transform(X_train)
    X_test_processed = preprocessor_best.transform(X_test)
    
    rf_model.fit(X_train_processed, y_train)
    y_pred_proba = rf_model.predict_proba(X_test_processed)
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
    cv_scores_d3.append(custom_mse)
    print(f"  Fold {fold_idx}: MSE = {custom_mse:.2f}")

mean_mse_d3 = np.mean(cv_scores_d3)
std_mse_d3 = np.std(cv_scores_d3)

print(f"\nMean MSE: {mean_mse_d3:.2f} ± {std_mse_d3:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - mean_mse_d3:.2f}")

log_experiment(
    experiment_id="D3",
    model_name="Random Forest",
    rationale="Non-boosted ensemble as baseline comparison",
    parameters=rf_model.get_params(),
    cv_scores=cv_scores_d3,
    mean_score=mean_mse_d3,
    std_score=std_mse_d3,
    additional_notes=f"Improvement: {mean_mse_baseline - mean_mse_d3:.2f}"
)


## Model Architecture Summary


In [ ]:
# Show all experiment results so far
print("="*70)
print("ALL EXPERIMENTS SUMMARY")
print("="*70)
all_summary = show_experiment_summary()
print(all_summary.to_string(index=False))
print("="*70)

# Identify top 3 models for hyperparameter tuning
top3_models = all_summary.head(3)
print(f"\nTop 3 Models for Hyperparameter Tuning:")
for idx, row in top3_models.iterrows():
    print(f"{row['Experiment_ID']}: {row['Model']} - MSE: {row['Mean_MSE']:.2f}")


---
# Phase 4: Hyperparameter Optimization

**Strategy**: Tune the top-performing models systematically


## Experiment E1: XGBoost Hyperparameter Tuning

**Rationale**: Systematically search for optimal learning rate, depth, and regularization


In [ ]:
# XGBoost hyperparameter grid search
print("Experiment E1: XGBoost Hyperparameter Tuning")

param_grid_xgb = [
    # Test 1: Deeper trees with more regularization
    {'max_depth': 10, 'learning_rate': 0.03, 'n_estimators': 400, 'reg_lambda': 2, 'subsample': 0.8},
    # Test 2: Shallower trees with higher learning rate
    {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 200, 'reg_lambda': 1, 'subsample': 0.9},
    # Test 3: Balanced approach
    {'max_depth': 8, 'learning_rate': 0.05, 'n_estimators': 350, 'reg_lambda': 1.5, 'subsample': 0.85},
    # Test 4: More estimators with lower learning rate
    {'max_depth': 7, 'learning_rate': 0.02, 'n_estimators': 500, 'reg_lambda': 1, 'subsample': 0.9},
]

best_xgb_score = float('inf')
best_xgb_params = None

for idx, params in enumerate(param_grid_xgb, 1):
    print(f"\nTesting XGBoost config {idx}/{len(param_grid_xgb)}: {params}")
    
    model = XGBClassifier(
        objective='multi:softprob',
        eval_metric='mlogloss',
        num_class=NUM_CLASSES,
        use_label_encoder=False,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        tree_method='hist',
        **params
    )
    
    cv_scores = []
    for train_index, test_index in skf.split(X_best, y_best):
        X_train, X_test = X_best.iloc[train_index], X_best.iloc[test_index]
        y_train, y_test = y_best.iloc[train_index], y_best.iloc[test_index]
        
        X_train_processed = preprocessor_best.fit_transform(X_train)
        X_test_processed = preprocessor_best.transform(X_test)
        
        model.fit(X_train_processed, y_train, verbose=False)
        y_pred_proba = model.predict_proba(X_test_processed)
        
        custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
        cv_scores.append(custom_mse)
    
    mean_score = np.mean(cv_scores)
    std_score = np.std(cv_scores)
    print(f"  Mean MSE: {mean_score:.2f} ± {std_score:.2f}")
    
    log_experiment(
        experiment_id=f"E1_{idx}",
        model_name=f"XGBoost Tuned {idx}",
        rationale=f"Hyperparameter config {idx}",
        parameters=params,
        cv_scores=cv_scores,
        mean_score=mean_score,
        std_score=std_score
    )
    
    if mean_score < best_xgb_score:
        best_xgb_score = mean_score
        best_xgb_params = params

print(f"\n{'='*70}")
print(f"Best XGBoost Config: {best_xgb_params}")
print(f"Best XGBoost MSE: {best_xgb_score:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - best_xgb_score:.2f}")


## Experiment E2: LightGBM Hyperparameter Tuning

**Rationale**: Optimize LightGBM-specific parameters (num_leaves, min_child_samples)


In [ ]:
# LightGBM hyperparameter grid search
print("Experiment E2: LightGBM Hyperparameter Tuning")

param_grid_lgbm = [
    # Test 1: More leaves with regularization
    {'num_leaves': 127, 'learning_rate': 0.03, 'n_estimators': 400, 'reg_lambda': 2, 'min_child_samples': 30},
    # Test 2: Fewer leaves, higher learning rate
    {'num_leaves': 31, 'learning_rate': 0.1, 'n_estimators': 200, 'reg_lambda': 1, 'min_child_samples': 20},
    # Test 3: Balanced approach
    {'num_leaves': 63, 'learning_rate': 0.05, 'n_estimators': 350, 'reg_lambda': 1.5, 'min_child_samples': 25},
    # Test 4: Conservative with more estimators
    {'num_leaves': 50, 'learning_rate': 0.02, 'n_estimators': 500, 'reg_lambda': 1, 'min_child_samples': 20},
]

best_lgbm_score = float('inf')
best_lgbm_params = None

for idx, params in enumerate(param_grid_lgbm, 1):
    print(f"\nTesting LightGBM config {idx}/{len(param_grid_lgbm)}: {params}")
    
    model = LGBMClassifier(
        objective='multiclass',
        num_class=NUM_CLASSES,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1,
        **params
    )
    
    cv_scores = []
    for train_index, test_index in skf.split(X_best, y_best):
        X_train, X_test = X_best.iloc[train_index], X_best.iloc[test_index]
        y_train, y_test = y_best.iloc[train_index], y_best.iloc[test_index]
        
        X_train_processed = preprocessor_best.fit_transform(X_train)
        X_test_processed = preprocessor_best.transform(X_test)
        
        model.fit(X_train_processed, y_train)
        y_pred_proba = model.predict_proba(X_test_processed)
        
        custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
        cv_scores.append(custom_mse)
    
    mean_score = np.mean(cv_scores)
    std_score = np.std(cv_scores)
    print(f"  Mean MSE: {mean_score:.2f} ± {std_score:.2f}")
    
    log_experiment(
        experiment_id=f"E2_{idx}",
        model_name=f"LightGBM Tuned {idx}",
        rationale=f"Hyperparameter config {idx}",
        parameters=params,
        cv_scores=cv_scores,
        mean_score=mean_score,
        std_score=std_score
    )
    
    if mean_score < best_lgbm_score:
        best_lgbm_score = mean_score
        best_lgbm_params = params

print(f"\n{'='*70}")
print(f"Best LightGBM Config: {best_lgbm_params}")
print(f"Best LightGBM MSE: {best_lgbm_score:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - best_lgbm_score:.2f}")


In [ ]:
# CatBoost hyperparameter grid search
print("Experiment E3: CatBoost Hyperparameter Tuning")

param_grid_catboost = [
    # Test 1: Deeper trees
    {'depth': 10, 'learning_rate': 0.03, 'iterations': 400, 'l2_leaf_reg': 5},
    # Test 2: Shallower trees, higher LR
    {'depth': 6, 'learning_rate': 0.1, 'iterations': 200, 'l2_leaf_reg': 3},
    # Test 3: Balanced
    {'depth': 8, 'learning_rate': 0.05, 'iterations': 350, 'l2_leaf_reg': 4},
    # Test 4: More iterations
    {'depth': 7, 'learning_rate': 0.02, 'iterations': 500, 'l2_leaf_reg': 3},
]

best_catboost_score = float('inf')
best_catboost_params = None

for idx, params in enumerate(param_grid_catboost, 1):
    print(f"\nTesting CatBoost config {idx}/{len(param_grid_catboost)}: {params}")
    
    model = CatBoostClassifier(
        loss_function='MultiClass',
        random_state=RANDOM_STATE,
        verbose=False,
        thread_count=-1,
        **params
    )
    
    cv_scores = []
    for train_index, test_index in skf.split(X_best, y_best):
        X_train, X_test = X_best.iloc[train_index], X_best.iloc[test_index]
        y_train, y_test = y_best.iloc[train_index], y_best.iloc[test_index]
        
        preprocessor_catboost = ColumnTransformer(
            transformers=[
                ('num', Pipeline([
                     ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
                     ('scaler', StandardScaler())]), NUMERICAL_FEATURES_BEST)
            ], remainder='passthrough'
        )
        
        X_train_processed = preprocessor_catboost.fit_transform(X_train)
        X_test_processed = preprocessor_catboost.transform(X_test)
        
        num_numerical = len(NUMERICAL_FEATURES_BEST)
        cat_indices = list(range(num_numerical, X_train_processed.shape[1]))
        
        model.fit(X_train_processed, y_train, cat_features=cat_indices)
        y_pred_proba = model.predict_proba(X_test_processed)
        
        custom_mse, _, _ = calculate_expected_payout_mse(y_test, y_pred_proba)
        cv_scores.append(custom_mse)
    
    mean_score = np.mean(cv_scores)
    std_score = np.std(cv_scores)
    print(f"  Mean MSE: {mean_score:.2f} ± {std_score:.2f}")
    
    log_experiment(
        experiment_id=f"E3_{idx}",
        model_name=f"CatBoost Tuned {idx}",
        rationale=f"Hyperparameter config {idx}",
        parameters=params,
        cv_scores=cv_scores,
        mean_score=mean_score,
        std_score=std_score
    )
    
    if mean_score < best_catboost_score:
        best_catboost_score = mean_score
        best_catboost_params = params

print(f"\n{'='*70}")
print(f"Best CatBoost Config: {best_catboost_params}")
print(f"Best CatBoost MSE: {best_catboost_score:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - best_catboost_score:.2f}")


---
# Phase 5: Ensemble Methods

**Strategy**: Combine predictions from multiple models to reduce variance


## Experiment F1: Simple Averaging Ensemble

**Rationale**: Average predictions from top 3 models to reduce variance


In [ ]:
print("Experiment F1: Simple Averaging Ensemble")

# Create best models based on hyperparameter tuning
best_xgb = XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    num_class=NUM_CLASSES,
    use_label_encoder=False,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    tree_method='hist',
    **best_xgb_params
)

best_lgbm = LGBMClassifier(
    objective='multiclass',
    num_class=NUM_CLASSES,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=-1,
    **best_lgbm_params
)

best_catboost = CatBoostClassifier(
    loss_function='MultiClass',
    random_state=RANDOM_STATE,
    verbose=False,
    thread_count=-1,
    **best_catboost_params
)

cv_scores_f1 = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_best, y_best), 1):
    X_train, X_test = X_best.iloc[train_index], X_best.iloc[test_index]
    y_train, y_test = y_best.iloc[train_index], y_best.iloc[test_index]
    
    # XGBoost and LightGBM
    X_train_processed = preprocessor_best.fit_transform(X_train)
    X_test_processed = preprocessor_best.transform(X_test)
    
    best_xgb.fit(X_train_processed, y_train, verbose=False)
    xgb_proba = best_xgb.predict_proba(X_test_processed)
    
    best_lgbm.fit(X_train_processed, y_train)
    lgbm_proba = best_lgbm.predict_proba(X_test_processed)
    
    # CatBoost
    preprocessor_catboost = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                 ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
                 ('scaler', StandardScaler())]), NUMERICAL_FEATURES_BEST)
        ], remainder='passthrough'
    )
    
    X_train_cat = preprocessor_catboost.fit_transform(X_train)
    X_test_cat = preprocessor_catboost.transform(X_test)
    
    num_numerical = len(NUMERICAL_FEATURES_BEST)
    cat_indices = list(range(num_numerical, X_train_cat.shape[1]))
    
    best_catboost.fit(X_train_cat, y_train, cat_features=cat_indices)
    catboost_proba = best_catboost.predict_proba(X_test_cat)
    
    # Simple average ensemble
    ensemble_proba = (xgb_proba + lgbm_proba + catboost_proba) / 3
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, ensemble_proba)
    cv_scores_f1.append(custom_mse)
    print(f"  Fold {fold_idx}: MSE = {custom_mse:.2f}")

mean_mse_f1 = np.mean(cv_scores_f1)
std_mse_f1 = np.std(cv_scores_f1)

print(f"\nMean MSE: {mean_mse_f1:.2f} ± {std_mse_f1:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - mean_mse_f1:.2f}")

log_experiment(
    experiment_id="F1",
    model_name="Ensemble (XGB+LGBM+CAT avg)",
    rationale="Average of top 3 tuned models",
    parameters={"models": ["XGBoost", "LightGBM", "CatBoost"], "method": "simple_average"},
    cv_scores=cv_scores_f1,
    mean_score=mean_mse_f1,
    std_score=std_mse_f1,
    additional_notes=f"Improvement: {mean_mse_baseline - mean_mse_f1:.2f}"
)


## Experiment F2: Weighted Ensemble

**Rationale**: Weight models based on their individual performance


In [ ]:
print("Experiment F2: Weighted Ensemble")

# Calculate weights inversely proportional to MSE
total_error = best_xgb_score + best_lgbm_score + best_catboost_score
w_xgb = (1/best_xgb_score) / ((1/best_xgb_score) + (1/best_lgbm_score) + (1/best_catboost_score))
w_lgbm = (1/best_lgbm_score) / ((1/best_xgb_score) + (1/best_lgbm_score) + (1/best_catboost_score))
w_catboost = (1/best_catboost_score) / ((1/best_xgb_score) + (1/best_lgbm_score) + (1/best_catboost_score))

print(f"Weights: XGB={w_xgb:.3f}, LGBM={w_lgbm:.3f}, CAT={w_catboost:.3f}")

cv_scores_f2 = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_best, y_best), 1):
    X_train, X_test = X_best.iloc[train_index], X_best.iloc[test_index]
    y_train, y_test = y_best.iloc[train_index], y_best.iloc[test_index]
    
    # Get predictions from all three models (same as F1)
    X_train_processed = preprocessor_best.fit_transform(X_train)
    X_test_processed = preprocessor_best.transform(X_test)
    
    best_xgb.fit(X_train_processed, y_train, verbose=False)
    xgb_proba = best_xgb.predict_proba(X_test_processed)
    
    best_lgbm.fit(X_train_processed, y_train)
    lgbm_proba = best_lgbm.predict_proba(X_test_processed)
    
    preprocessor_catboost = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                 ('log', FunctionTransformer(lambda x: np.log1p(x), validate=False)),
                 ('scaler', StandardScaler())]), NUMERICAL_FEATURES_BEST)
        ], remainder='passthrough'
    )
    
    X_train_cat = preprocessor_catboost.fit_transform(X_train)
    X_test_cat = preprocessor_catboost.transform(X_test)
    
    num_numerical = len(NUMERICAL_FEATURES_BEST)
    cat_indices = list(range(num_numerical, X_train_cat.shape[1]))
    
    best_catboost.fit(X_train_cat, y_train, cat_features=cat_indices)
    catboost_proba = best_catboost.predict_proba(X_test_cat)
    
    # Weighted ensemble
    ensemble_proba = w_xgb * xgb_proba + w_lgbm * lgbm_proba + w_catboost * catboost_proba
    
    custom_mse, _, _ = calculate_expected_payout_mse(y_test, ensemble_proba)
    cv_scores_f2.append(custom_mse)
    print(f"  Fold {fold_idx}: MSE = {custom_mse:.2f}")

mean_mse_f2 = np.mean(cv_scores_f2)
std_mse_f2 = np.std(cv_scores_f2)

print(f"\nMean MSE: {mean_mse_f2:.2f} ± {std_mse_f2:.2f}")
print(f"Improvement vs Baseline: {mean_mse_baseline - mean_mse_f2:.2f}")

log_experiment(
    experiment_id="F2",
    model_name="Ensemble (Weighted)",
    rationale="Performance-weighted average of top 3 models",
    parameters={"models": ["XGBoost", "LightGBM", "CatBoost"], "method": "weighted", 
                "weights": f"XGB={w_xgb:.3f}, LGBM={w_lgbm:.3f}, CAT={w_catboost:.3f}"},
    cv_scores=cv_scores_f2,
    mean_score=mean_mse_f2,
    std_score=std_mse_f2,
    additional_notes=f"Improvement: {mean_mse_baseline - mean_mse_f2:.2f}"
)


---
# Phase 6: Final Model Selection

**Strategy**: Select the best performing model and document complete pipeline


In [ ]:
# Final comprehensive summary
print("="*80)
print(" "*25 + "FINAL EXPERIMENT SUMMARY")
print("="*80)

final_summary = show_experiment_summary()
print(final_summary.head(10).to_string(index=False))
print("="*80)

# Select best model
best_model_row = final_summary.iloc[0]
print(f"\n🏆 BEST MODEL: {best_model_row['Model']}")
print(f"   Experiment ID: {best_model_row['Experiment_ID']}")
print(f"   Mean MSE: {best_model_row['Mean_MSE']:.2f} ± {best_model_row['Std_MSE']:.2f}")
print(f"   Improvement over Baseline: {mean_mse_baseline - best_model_row['Mean_MSE']:.2f}")
print(f"   Percentage Improvement: {((mean_mse_baseline - best_model_row['Mean_MSE'])/mean_mse_baseline * 100):.1f}%")
print(f"\nRationale: {best_model_row['Rationale']}")


## Model Performance Visualization


In [ ]:
# Visualize top 10 models
fig, ax = plt.subplots(figsize=(12, 6))

top_models = final_summary.head(10)
y_pos = np.arange(len(top_models))

bars = ax.barh(y_pos, top_models['Mean_MSE'], xerr=top_models['Std_MSE'], 
               color=['green' if i == 0 else 'steelblue' for i in range(len(top_models))])

ax.set_yticks(y_pos)
ax.set_yticklabels(top_models['Experiment_ID'] + ': ' + top_models['Model'])
ax.invert_yaxis()
ax.set_xlabel('Payout MSE (lower is better)')
ax.set_title('Top 10 Model Performances')
ax.axvline(x=mean_mse_baseline, color='red', linestyle='--', label=f'Baseline: {mean_mse_baseline:.2f}')
ax.legend()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()


## Key Findings and Insights

Based on the experiments:


In [ ]:
print("KEY FINDINGS:")
print("="*70)

# Feature engineering insights
feature_exps = [exp for exp in experiment_results if exp['Experiment_ID'] in ['A1', 'A2', 'B1', 'C1', 'C2']]
if feature_exps:
    best_feature = min(feature_exps, key=lambda x: x['Mean_MSE'])
    print(f"\n1. FEATURE ENGINEERING:")
    print(f"   Most impactful features: {best_feature['Experiment_ID']} - {best_feature['Model']}")
    print(f"   Improvement: {mean_mse_baseline - best_feature['Mean_MSE']:.2f}")

# Model architecture insights
model_exps = [exp for exp in experiment_results if exp['Experiment_ID'] in ['D1', 'D2', 'D3']]
if model_exps:
    best_arch = min(model_exps, key=lambda x: x['Mean_MSE'])
    print(f"\n2. MODEL ARCHITECTURE:")
    print(f"   Best base model: {best_arch['Model']}")
    print(f"   MSE: {best_arch['Mean_MSE']:.2f}")

# Hyperparameter tuning insights
tuned_exps = [exp for exp in experiment_results if exp['Experiment_ID'].startswith('E')]
if tuned_exps:
    best_tuned = min(tuned_exps, key=lambda x: x['Mean_MSE'])
    print(f"\n3. HYPERPARAMETER TUNING:")
    print(f"   Best tuned model: {best_tuned['Model']}")
    print(f"   MSE: {best_tuned['Mean_MSE']:.2f}")

# Ensemble insights
ensemble_exps = [exp for exp in experiment_results if exp['Experiment_ID'] in ['F1', 'F2']]
if ensemble_exps:
    best_ensemble = min(ensemble_exps, key=lambda x: x['Mean_MSE'])
    print(f"\n4. ENSEMBLE METHODS:")
    print(f"   Best ensemble: {best_ensemble['Model']}")
    print(f"   MSE: {best_ensemble['Mean_MSE']:.2f}")

print(f"\n{'='*70}")
print(f"OVERALL BEST: {best_model_row['Model']}")
print(f"Final MSE: {best_model_row['Mean_MSE']:.2f} (vs Baseline: {mean_mse_baseline:.2f})")
print(f"Absolute Improvement: {mean_mse_baseline - best_model_row['Mean_MSE']:.2f}")
print(f"Relative Improvement: {((mean_mse_baseline - best_model_row['Mean_MSE'])/mean_mse_baseline * 100):.1f}%")
print("="*70)


## Save Experiment Results


In [ ]:
# Save all experiment results to CSV for reference
df_all_results = pd.DataFrame(experiment_results)
df_all_results.to_csv('experiment_results.csv', index=False)
print("Experiment results saved to 'experiment_results.csv'")

# Save the final summary
final_summary.to_csv('model_comparison_summary.csv', index=False)
print("Model comparison saved to 'model_comparison_summary.csv'")


---
# Complete Pipeline for Best Model

Below is the complete, reproducible code for the best performing model:


In [ ]:
print("""
COMPLETE REPRODUCIBLE PIPELINE FOR BEST MODEL
=============================================

The best model configuration will be determined after running all experiments.
This cell will be updated with the exact code to reproduce the best results.

Key components:
1. Feature engineering (exact transformations)
2. Preprocessing pipeline
3. Model configuration with optimal hyperparameters
4. Training procedure
5. Evaluation code

Random seed: 42 (for reproducibility)
Cross-validation: 3-fold stratified
Metric: Custom Payout MSE

After running this notebook, copy the best configuration to the main notebook.
""")
